In [ ]:
proj_list = [
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'lcms',
    'libpng', 'libssh',
    'libxml2', 'pcre2',
    'proj4', 're2',
    'sqlite3', 'vorbis',
    'woff2', 'wpantund'
]

In [ ]:
target_project = 0

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [ ]:
gc.collect()
torch.cuda.empty_cache()

print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
prefix_np, postfix_np, label_np, label_len_np = data.getTrainData(proj_list, proj_list[target_project])

In [ ]:
test_prefix, test_postfix, test_label, test_label_len = data.getTestData(proj_list[target_project])

In [ ]:
train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
    prefix_np, postfix_np, label_np, test_size = 0.2, random_state = 43
)

In [ ]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, train_postfix,
        val_prefix, val_postfix,
        test_prefix, test_postfix,
        train_label, val_label, test_label,
        batch_size=1000
    )

In [ ]:
overall_title = 'developing_1'

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/'+overall_title+'/tests')

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
# ====================
# set parameters here
# ====================

title = proj_list[target_project] + '_' + overall_title + '_1'
epochs = 40

max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.001
weight_decay = 0.0

embed_dim = 128
hidden_size = 200
n_layers = 2
output_size = max(token_choices) + 2
dropout = 0.0
max_length = max_len
input_size = max(token_choices) + 2
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "CEL"

teacher_forcing_ratio = 100.0

In [ ]:
trainer.set_seed(42)

loss_fn, prefix_pack, postfix_pack, attn_pack = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device
)

In [ ]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack,
    teacher_forcing_ratio=teacher_forcing_ratio
)

end_time = (timeit.default_timer() - start_time)

In [ ]:
mu.saveModel(overall_title, title, prefix_pack, postfix_pack, attn_pack)

In [ ]:
# model = mu.getModel(title)
# print(model)

In [ ]:
loss, acc = tester.test(
    test_dataloader=test_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack
)

In [ ]:
with open('../stat/'+title, 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [ ]:
# mu.graphModel(train_dataloader, model, writer)